# Benchmark CNN Keras model

In [ ]:
import os
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import random
import shutil


data_dir = '../data/'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
sample_submission = pd.read_csv(os.path.join(data_dir,'sample_submission.csv'))
sample_submission.head(2)

,file,species
0,0021e90e4.png,Sugar beet
1,003d61042.png,Sugar beet


In [8]:
CATEGORIES = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent',
              'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']
NUM_CATEGORIES = len(CATEGORIES)



In [9]:
for category in CATEGORIES:
    print('{} {} images'.format(category, len(os.listdir(os.path.join(train_dir, category)))))

Black-grass 169 images
Charlock 250 images
Cleavers 184 images
Common Chickweed 392 images
Common wheat 142 images
Fat Hen 304 images
Loose Silky-bent 420 images
Maize 142 images
Scentless Mayweed 331 images
Shepherds Purse 148 images
Small-flowered Cranesbill 318 images
Sugar beet 247 images


In [10]:
trainData = []
for categoryID,category in enumerate(CATEGORIES):
    for file in os.listdir(os.path.join(train_dir,category)):
        trainData.append(['train/{}/{}'.format(category, file), categoryID, category])
trainData = pd.DataFrame(trainData, columns=['fileName', 'categoryID', 'category'])
trainData.head(2)


,fileName,categoryID,category
0,train/Black-grass/5a1295fb4.png,0,Black-grass
1,train/Black-grass/d8afd58f3.png,0,Black-grass


In [11]:
trainData.shape

(3047, 3)

In [13]:
#  split validation data from  train data
os.mkdir('../data/val/')
for category in CATEGORIES:
    os.mkdir('../data/val/' + category)
    name = os.listdir('../data/train/' + category)
    random.shuffle(name) 
    toVal = name[:int(len(name) * 0.2)] # split 20%
    for file in toVal:
        shutil.move(os.path.join('../data/train/', category, file), os.path.join('../data/val/', category))    
        # shutil - High level file operations        
        # https://docs.python.org/2/library/shutil.html

FileExistsError: [Errno 17] File exists: '../data/val/'

In [14]:
# https://www.kaggle.com/dingkun/xception-model-training-pipeline-lb-0-9798


trainDataGenerator = ImageDataGenerator(rescale=1. / 255,  # Normalization
                                        rotation_range=50,
                                        width_shift_range=0.2,  # fraction of total width
                                        height_shift_range=0.2,
                                        shear_range=0.2,
                                        zoom_range=0.2,
                                        horizontal_flip=True,
                                        vertical_flip=True)

# flow_from_directory(directory): Takes the path to a directory, 
# and generates batches of augmented/normalized data. Yields batches indefinitely, 
# in an infinite loop.
trainGenerated = trainDataGenerator.flow_from_directory('../data/train',
                                                        # The dimensions to which all images found will be resized.
                                                        target_size=(48, 48),
                                                        batch_size=16,
                                                        class_mode='categorical',
                                                        shuffle=True
                                                        )

valDataGenerator = ImageDataGenerator(rescale=1. / 255)

valGenerated = valDataGenerator.flow_from_directory('../data/val',
                                                    target_size=(48, 48),
                                                    batch_size=16,
                                                    class_mode='categorical',
                                                    shuffle=True
                                                    )



Found 3046 images belonging to 12 classes.
Found 756 images belonging to 12 classes.


In [15]:
from keras.callbacks import TensorBoard

tensorboard = TensorBoard('../logs')

In [16]:
from keras.layers import Dropout, Input, Dense, Activation,GlobalMaxPooling2D, BatchNormalization, Flatten, Conv2D, MaxPooling2D
from keras.models import Model, load_model
from keras.optimizers import Adam
from keras.models import Sequential



In [17]:
model = Sequential()
model.add(Conv2D(16, (3, 3), input_shape=(48, 48, 3)))
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))
model.add(Conv2D(16, (3, 3)))
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))
model.add(GlobalMaxPooling2D())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(12, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-4), metrics=['acc'])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 16)        448       
_________________________________________________________________
batch_normalization_1 (Batch (None, 46, 46, 16)        64        
_________________________________________________________________
activation_1 (Activation)    (None, 46, 46, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 44, 44, 16)        2320      
_________________________________________________________________
batch_normalization_2 (Batch (None, 44, 44, 16)        64        
_________________________________________________________________
activation_2 (Activation)    (None, 44, 44, 16)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 22, 16)        0         
__________

In [5]:
from keras.callbacks import LearningRateScheduler, EarlyStopping
from keras.callbacks import ModelCheckpoint
from IPython.display import display 
from PIL import Image
batch_size = 64
lr = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
earlystop = EarlyStopping(patience=20)
modelsave = ModelCheckpoint(
    filepath='../checkpoint/weights-{epoch:02d}-{val_loss:.2f}.h5', 
    save_best_only=True, 
    verbose=1)

model.fit_generator(trainGenerated,
                    steps_per_epoch=batch_size,
                    epochs=200, 
                    validation_data=valGenerated,
                    callbacks=[modelsave, tensorboard,earlystop, lr],
                    workers = 4
                    )
model.save('CNNbenchmark.h5')

NameError: name 'model' is not defined

In [20]:
import glob
from PIL import Image, ImageOps

z = glob.glob('../data/TEST1/test/*.png')
test_imgs = []
names = []
for fn in z:
    if fn[-3:] != 'png':
        continue
    names.append(fn.split('/')[-1])
    new_img = Image.open(fn)
    test_img = ImageOps.fit(new_img, (48, 48), Image.ANTIALIAS).convert('RGB')
    test_imgs.append(test_img)
model = load_model('../model/CNNbenchmark.h5')

In [21]:
timgs = np.array([np.array(im) for im in test_imgs])
testX = timgs.reshape(timgs.shape[0], 48, 48, 3) / 255

In [22]:
yhat = model.predict(testX)

In [23]:
preds = []
for i in range(len(yhat)):
    pos = np.argmax(yhat[i])
    preds.append(CATEGORIES[pos])

In [24]:
df = pd.DataFrame(data={'file': names, 'species': preds})

In [25]:
df.head(5)

,file,species
0,1b490196c.png,Small-flowered Cranesbill
1,85431c075.png,Loose Silky-bent
2,506347cfe.png,Scentless Mayweed
3,7f46a71db.png,Scentless Mayweed
4,668c1007c.png,Charlock


In [26]:
df.tail(4)

,file,species
790,78b1bf91a.png,Sugar beet
791,8cf2e3e6c.png,Common Chickweed
792,948cdb277.png,Cleavers
793,24c94a6ca.png,Charlock


In [30]:
df_sort = df.sort_values(by=['file'])
df_sort.to_csv('../submission/benchmarResults.csv', index=False)

# start parameter tuning

change dimentation from 48 to 128

In [ ]:
dim128 = 128
model_dim128 = Sequential()
model_dim128.add(Conv2D(16, (3, 3), input_shape=(dim128, dim128, 3)))
model_dim128.add(BatchNormalization(axis=3))
model_dim128.add(Activation('relu'))
model_dim128.add(Conv2D(16, (3, 3)))
model_dim128.add(BatchNormalization(axis=3))
model_dim128.add(Activation('relu'))
model_dim128.add(MaxPooling2D((2, 2), strides=(2, 2)))
model_dim128.add(Conv2D(32, (3, 3)))
model_dim128.add(BatchNormalization(axis=3))
model_dim128.add(Activation('relu'))
model_dim128.add(Conv2D(32, (3, 3)))
model_dim128.add(BatchNormalization(axis=3))
model_dim128.add(Activation('relu'))
model_dim128.add(GlobalMaxPooling2D())

model_dim128.add(Dense(64, activation='relu'))
model_dim128.add(Dropout(0.5))
model_dim128.add(Dense(32, activation='relu'))
model_dim128.add(Dropout(0.5))
model_dim128.add(Dense(12, activation='softmax'))
model_dim128.summary()
model_dim128.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-4), metrics=['acc'])

1. change dimentation from 48 to 128
2. increase number of filters

In [ ]:
dim128 = 128
model_dim128_deep = Sequential()
model_dim128_deep.add(Conv2D(16, (7, 7), strides=(2,2),input_shape=(dim128, dim128, 3))) #changed
model_dim128_deep.add(BatchNormalization(axis=3))
model_dim128_deep.add(Activation('relu'))
# model_dim128_deep.add(Conv2D(16, (3, 3)))
# model_dim128_deep.add(BatchNormalization(axis=3))
# model_dim128_deep.add(Activation('relu'))
#model_dim128_deep.add(MaxPooling2D((2, 2), strides=(2, 2)))
model_dim128_deep.add(Conv2D(32, (3, 3)))
model_dim128_deep.add(BatchNormalization(axis=3))
model_dim128_deep.add(Activation('relu'))
model_dim128_deep.add(Conv2D(64, (3, 3)))
model_dim128_deep.add(BatchNormalization(axis=3))
model_dim128_deep.add(Activation('relu'))
model_dim128_deep.add(MaxPooling2D((2, 2), strides=(2, 2)))
model_dim128_deep.add(Conv2D(64, (3, 3)))
model_dim128_deep.add(BatchNormalization(axis=3))
model_dim128_deep.add(Activation('relu'))
#model_dim128_deep.add(Conv2D(32, (3, 3)))
#model_dim128_deep.add(BatchNormalization(axis=3))
#model_dim128_deep.add(Activation('relu'))
model_dim128_deep.add(GlobalMaxPooling2D())

model_dim128_deep.add(Dense(64, activation='relu'))
model_dim128_deep.add(Dropout(0.5))
model_dim128_deep.add(Dense(32, activation='relu'))
model_dim128_deep.add(Dropout(0.5))
model_dim128_deep.add(Dense(12, activation='softmax'))
model_dim128_deep.summary()
model_dim128_deep.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-4), metrics=['acc'])

